In [26]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

df_train = pd.read_csv('dataset/breast-cancer-wisconsin/train_data.csv')
X_train = df_train.drop('diagnosis',axis=1)
y_train = df_train['diagnosis']

feature_names = X_train.columns

df_test = pd.read_csv('dataset/breast-cancer-wisconsin/test_case/test_case_mlp_lower.csv')
X_test = df_test.drop('diagnosis',axis=1)
y_test = df_test['diagnosis']

test_row_count = df_test.shape[0]

from sklearn.preprocessing import StandardScaler
transform = StandardScaler()
X_train = transform.fit_transform(X_train)
X_test = transform.transform(X_test)

noise_std = 0.2
np.random.seed(0)
X_test_noisy = X_test + noise_std * np.random.randn(*X_test.shape)

In [27]:
import joblib
model = joblib.load('saved_model/breast-cancer-wisconsin/MLP.pkl')

y_pred = model.predict(X_test_noisy)
y_pred_proba = model.predict_proba(X_test_noisy)

In [28]:
import lime
from lime import lime_tabular
explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=feature_names,
    class_names=['Benign','Malignant'],
    mode='classification'
)

In [29]:
explanations = [] 

# First loop over idx, generate explanation, and store the results in explanations
for idx in range(test_row_count):
    exp = explainer.explain_instance(
            data_row=X_test_noisy[idx], 
            predict_fn=model.predict_proba,
            num_features = 30
        )
    explanations.append(exp)

In [30]:
for k in range(3, 4):
    test_data = pd.read_csv('dataset/breast-cancer-wisconsin/test_case/test_case_mlp_lower_noise.csv')
    bound_standardized = pd.read_csv('dataset/breast-cancer-wisconsin/bound_standardized.csv')
    for idx in range(test_row_count):
        exp = explanations[idx] # Use the previously stored explanation
        mapa = exp.as_map().values()
        lista = list(mapa)[0]
        for j in range(k):
            s = lista[j]
            test_data.iat[idx,s[0]+1]=bound_standardized.iat[0,s[0]]
    filename = 'explain_set_lower/breast-cancer-wisconsin/MLP/lime/noise/test_case/lime_explain_'+str(k)+'.csv'
    test_data.to_csv(filename,index=False)